# Bluebook for Bulldozers

## Preprocessing and Training

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA # not yet using
from sklearn.preprocessing import scale # not yet using
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, RandomizedSearchCV
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv(
    '../data/processed/exploratory-data-analysis.csv',
    low_memory=False
)

### Converting strings into categories

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372915 entries, 0 to 372914
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   372915 non-null  int64  
 1   SalePrice                 372915 non-null  float64
 2   MachineID                 372915 non-null  int64  
 3   ModelID                   372915 non-null  int64  
 4   datasource                372915 non-null  int64  
 5   auctioneerID              355456 non-null  float64
 6   YearMade                  372915 non-null  int64  
 7   MachineHoursCurrentMeter  130277 non-null  float64
 8   UsageBand                 68017 non-null   object 
 9   fiModelDesc               372915 non-null  object 
 10  fiBaseModel               372915 non-null  object 
 11  fiSecondaryDesc           250059 non-null  object 
 12  fiModelSeries             49026 non-null   object 
 13  fiModelDescriptor         70124 non-null   o

In [4]:
#  find the columns that contain strings, and turn them into categories
for label, content in df.items():
    if pd.api.types.is_string_dtype(content):
        df[label] = content.astype("category").cat.as_ordered()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372915 entries, 0 to 372914
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   SalesID                   372915 non-null  int64   
 1   SalePrice                 372915 non-null  float64 
 2   MachineID                 372915 non-null  int64   
 3   ModelID                   372915 non-null  int64   
 4   datasource                372915 non-null  int64   
 5   auctioneerID              355456 non-null  float64 
 6   YearMade                  372915 non-null  int64   
 7   MachineHoursCurrentMeter  130277 non-null  float64 
 8   UsageBand                 68017 non-null   category
 9   fiModelDesc               372915 non-null  category
 10  fiBaseModel               372915 non-null  category
 11  fiSecondaryDesc           250059 non-null  category
 12  fiModelSeries             49026 non-null   category
 13  fiModelDescriptor         701

In [6]:
df.state.cat.categories

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Unspecified', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'Washington DC', 'West Virginia', 'Wisconsin',
       'Wyoming'],
      dtype='object')

Great -- now the string types have been converted into categories

## Filling in missing values

### Fill numeric rows with median value

In [7]:
# Check which columns have null values
numeric_null_columns = []
for label, content in df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            numeric_null_columns.append(label)
numeric_null_columns

['auctioneerID', 'MachineHoursCurrentMeter']

In [8]:
df[numeric_null_columns].isna().sum()

auctioneerID                 17459
MachineHoursCurrentMeter    242638
dtype: int64

In [9]:
for column in numeric_null_columns:
    median = df[column].median()
    df[column].fillna(median, inplace=True)

In [10]:
df[numeric_null_columns].isna().sum()

auctioneerID                0
MachineHoursCurrentMeter    0
dtype: int64

Great -- these values have been successfully imputed!

### Fill in categorical values

In [11]:
# check for columns that are not numeric
for label, content in df.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

UsageBand
fiModelDesc
fiBaseModel
fiSecondaryDesc
fiModelSeries
fiModelDescriptor
ProductSize
fiProductClassDesc
state
ProductGroup
ProductGroupDesc
Drive_System
Enclosure
Forks
Pad_Type
Ride_Control
Stick
Transmission
Turbocharged
Blade_Extension
Blade_Width
Enclosure_Type
Engine_Horsepower
Hydraulics
Pushblock
Ripper
Scarifier
Tip_Control
Tire_Size
Coupler
Coupler_System
Grouser_Tracks
Hydraulics_Flow
Track_Type
Undercarriage_Pad_Width
Stick_Length
Thumb
Pattern_Changer
Grouser_Type
Backhoe_Mounting
Blade_Type
Travel_Controls
Differential_Type
Steering_Controls


In [12]:
# turn categorical values into numbers, and fill in missing
for label, content in df.items():
    if not pd.api.types.is_numeric_dtype(content):
        # add a binary column to indicate whether row had a missing value
        df[label + '_is_missing'] = pd.isnull(content)
        # turn categories into numbers, and add one becuase padans assigns NaNs a value of -1
        df[label] = pd.Categorical(content).codes + 1

In [13]:
pd.Categorical(df['Transmission']).codes

array([3, 0, 7, ..., 0, 0, 0], dtype=int8)

Great -- these values now have codes!

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372915 entries, 0 to 372914
Columns: 101 entries, SalesID to Steering_Controls_is_missing
dtypes: bool(44), float64(3), int16(4), int64(10), int8(40)
memory usage: 69.7 MB


In [15]:
df.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc',
       ...
       'Undercarriage_Pad_Width_is_missing', 'Stick_Length_is_missing',
       'Thumb_is_missing', 'Pattern_Changer_is_missing',
       'Grouser_Type_is_missing', 'Backhoe_Mounting_is_missing',
       'Blade_Type_is_missing', 'Travel_Controls_is_missing',
       'Differential_Type_is_missing', 'Steering_Controls_is_missing'],
      dtype='object', length=101)

In [16]:
df.isna().sum()

SalesID                         0
SalePrice                       0
MachineID                       0
ModelID                         0
datasource                      0
                               ..
Backhoe_Mounting_is_missing     0
Blade_Type_is_missing           0
Travel_Controls_is_missing      0
Differential_Type_is_missing    0
Steering_Controls_is_missing    0
Length: 101, dtype: int64

Nice -- we now have zero missing values

### Split the data to train and validation sets

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns='SalePrice'), 
    df.SalePrice,
    test_size=0.3, 
    stratify=None, # we may want to stratify imbalanced labels
    random_state=42
)

In [18]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((261040, 100), (261040,), (111875, 100), (111875,))

### Building an evaluation function
Since we will be testing many instances of many models, we should put together a `show_scores` function.

In [19]:
def rmsle(y_test, y_preds):
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_test)
    scores = {
        "Training...MAE": mean_absolute_error(y_train, train_preds),
        "Test.......MAE": mean_absolute_error(y_test, val_preds),
        "Training...RMLSE": rmsle(y_train, train_preds),
        "Test.......RMLSE": rmsle(y_test, val_preds),
        "Training...R2": r2_score(y_train, train_preds),
        "Test.......R2": r2_score(y_test, val_preds),
    }
    return scores

### Naive model implementation
Let's set up a quick model, on a subset of our data, and test it against our `show_scores` function

In [20]:
model = RandomForestRegressor(
    n_jobs=-1,
    random_state=42,
    max_samples=10000 # for better performance, since this is a massive dataset, we can train on samples.
)

In [21]:
%%time
model.fit(X_train, y_train)

CPU times: user 37.5 s, sys: 639 ms, total: 38.2 s
Wall time: 5 s


RandomForestRegressor(max_samples=10000, n_jobs=-1, random_state=42)

In [22]:
show_scores(model)

{'Training...MAE': 5395.372446789763,
 'Test.......MAE': 5579.017371477095,
 'Training...RMLSE': 0.23642824378258032,
 'Test.......RMLSE': 0.24545416628170555,
 'Training...R2': 0.8753352313858191,
 'Test.......R2': 0.865301689053226}

## Hyperparameter tuning with GridSearchCV

In [ ]:
%%time
param_grid = {
    "n_estimators": np.arange(10, 100, 10),
    "max_depth": [None, 3, 5, 10],
    "min_samples_split": np.arange(2, 20, 2),
    "min_samples_leaf": np.arange(1, 20, 2),
    "max_features": [0.5, 1, 'sqrt', 'auto'],
    "max_samples": [1000] # setting this for performance reasons
}

rs_model = RandomizedSearchCV(
    RandomForestRegressor(
        n_jobs=-1,
        random_state=42
        # we should consider definig a score attr?  one fitted to our desired metric?
        # https://youtu.be/0B5eIE_1vpU?t=3662
        # be sure to tune the greater_is_better param seen here:
        # https://youtu.be/0B5eIE_1vpU?t=5063
    ),
    param_distributions=param_grid,
    n_iter=2, # this is also set for performance reasons -- increase for greater accuracy!
    cv=5,
    verbose=True
)

# Fit the randomized search CV model!
rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
